## Proyecto final BOOTCAMP IRONHACK
###### Estrategias de e-COMERCE

COMPANY = GYMSHARK

### PASO 3. ETL
- website : https://eu.gymshark.com/

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

### DF Nº1: SALES 2023

In [49]:
# Importo el CSV de GYMSHARK SALES resultante del fake
sales_gymshark = pd.read_csv('sales_gymshark.concat.csv')

In [50]:
len(sales_gymshark)

410000

In [51]:
sales_gymshark.shape

(410000, 12)

In [52]:
sales_gymshark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410000 entries, 0 to 409999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    410000 non-null  int64  
 1   StockCode    410000 non-null  object 
 2   Description  410000 non-null  object 
 3   Size         410000 non-null  object 
 4   Quantity     410000 non-null  int64  
 5   InvoiceDate  410000 non-null  object 
 6   CustomerID   410000 non-null  int64  
 7   Country      410000 non-null  object 
 8   Rating       379989 non-null  float64
 9   Colour       409045 non-null  object 
 10  Price        410000 non-null  float64
 11  Category     410000 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 37.5+ MB


In [53]:
sales_gymshark.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Size', 'Quantity',
       'InvoiceDate', 'CustomerID', 'Country', 'Rating', 'Colour', 'Price',
       'Category'],
      dtype='object')

In [54]:
sales_gymshark.isnull().sum()

InvoiceNo          0
StockCode          0
Description        0
Size               0
Quantity           0
InvoiceDate        0
CustomerID         0
Country            0
Rating         30011
Colour           955
Price              0
Category           0
dtype: int64

In [55]:
# RELLENAR NULOS DE LAS COLUMNAS "Rating" Y "Colour"

# Rellenar valores nulos en la columna "Rating" con la mediana, son productos que nunca ha sido rankeados por los clientes.
sales_gymshark['Rating'] = sales_gymshark['Rating'].fillna(sales_gymshark['Rating'].median())


# Rellenar valores nulos en la columna "Colour" con "multicolored"
sales_gymshark['Colour'] = sales_gymshark['Colour'].fillna('multicolored')


In [56]:
sales_gymshark.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Size           0
Quantity       0
InvoiceDate    0
CustomerID     0
Country        0
Rating         0
Colour         0
Price          0
Category       0
dtype: int64

In [57]:
sales_gymshark.duplicated().sum()

0

In [58]:
# Convertir 'InvoiceDate' a formato de fecha y hora
sales_gymshark['InvoiceDate'] = pd.to_datetime(sales_gymshark['InvoiceDate'], format='%m-%d-%Y %A %H:%M')

# Separar la columna 'InvoiceDate' en fecha y hora
sales_gymshark['Day'] = sales_gymshark['InvoiceDate'].dt.day
sales_gymshark['Month'] = sales_gymshark['InvoiceDate'].dt.month
sales_gymshark['Year'] = sales_gymshark['InvoiceDate'].dt.year
sales_gymshark['Hour'] = sales_gymshark['InvoiceDate'].dt.strftime('%H:%M:%S')
sales_gymshark['DayOfWeek'] = sales_gymshark['InvoiceDate'].dt.dayofweek


# Mostrar las primeras filas
sales_gymshark.head()


,InvoiceNo,StockCode,Description,Size,Quantity,InvoiceDate,CustomerID,Country,Rating,Colour,Price,Category,Day,Month,Year,Hour,DayOfWeek
0,245,78278Z,Running 1/4 Zip,XXS,2,2021-06-22 02:58:00,2344,Canada,4.5,Plum Pink,38.442690,Women,22,6,2021,02:58:00,1
1,616,01000C,Lifting Lightweight 1/4 Zip Pullover,S,2,2021-05-23 22:31:00,54001,Canada,5.0,Lido Green,26.560404,Women,23,5,2021,22:31:00,6
2,453,68522Z,"Sport 5"" 2 In 1 Shorts",L,1,2021-02-09 00:38:00,96003,USA,3.9,Light Olive Green,29.858400,Women,9,2,2021,00:38:00,1
3,578,36909N,Crest Joggers,M,1,2021-05-08 15:54:00,34219,UK,4.4,Black,28.501200,Women,8,5,2021,15:54:00,5
4,863,53352X,Crew Socks 5pk,M,2,2021-03-27 22:38:00,17438,USA,4.7,Black,14.929200,Accesorios,27,3,2021,22:38:00,5


In [59]:
sales_gymshark.dtypes


InvoiceNo               int64
StockCode              object
Description            object
Size                   object
Quantity                int64
InvoiceDate    datetime64[ns]
CustomerID              int64
Country                object
Rating                float64
Colour                 object
Price                 float64
Category               object
Day                     int32
Month                   int32
Year                    int32
Hour                   object
DayOfWeek               int32
dtype: object

### PROCEDO A CREAR COLUMNAS QUE NECESITO PARA ANALISIS:

In [60]:
sales_gymshark['Total'] = sales_gymshark['Quantity'] * sales_gymshark['Price']

# Calcular el coste, la utilidad y los gastos operacionales
sales_gymshark['Cost'] = (sales_gymshark['Total'] * 0.35).round(2)
sales_gymshark['Profit'] = (sales_gymshark['Total'] * 0.13).round(2)
sales_gymshark['Operational Costs'] = (sales_gymshark['Total'] * 0.52).round(2)


In [61]:
# Convertir las columnas numéricas al formato deseado con coma como separador decimal
columns_to_format = ['Rating','Price', 'Total', 'Cost', 'Profit', 'Operational Costs']

for column in columns_to_format:
    # Convertir la columna al tipo de datos numérico
    sales_gymshark[column] = sales_gymshark[column].astype(float)
    # Aplicar el formato deseado con coma como separador decimal
    sales_gymshark[column] = sales_gymshark[column].map("{:,.2f}".format).str.replace('.', ',')

In [62]:
sales_gymshark.dtypes

InvoiceNo                     int64
StockCode                    object
Description                  object
Size                         object
Quantity                      int64
InvoiceDate          datetime64[ns]
CustomerID                    int64
Country                      object
Rating                       object
Colour                       object
Price                        object
Category                     object
Day                           int32
Month                         int32
Year                          int32
Hour                         object
DayOfWeek                     int32
Total                        object
Cost                         object
Profit                       object
Operational Costs            object
dtype: object

In [63]:
sales_gymshark

,InvoiceNo,StockCode,Description,Size,Quantity,InvoiceDate,CustomerID,Country,Rating,Colour,...,Category,Day,Month,Year,Hour,DayOfWeek,Total,Cost,Profit,Operational Costs
0,245,78278Z,Running 1/4 Zip,XXS,2,2021-06-22 02:58:00,2344,Canada,"4,50",Plum Pink,...,Women,22,6,2021,02:58:00,1,"76,89","26,91","10,00","39,98"
1,616,01000C,Lifting Lightweight 1/4 Zip Pullover,S,2,2021-05-23 22:31:00,54001,Canada,"5,00",Lido Green,...,Women,23,5,2021,22:31:00,6,"53,12","18,59","6,91","27,62"
2,453,68522Z,"Sport 5"" 2 In 1 Shorts",L,1,2021-02-09 00:38:00,96003,USA,"3,90",Light Olive Green,...,Women,9,2,2021,00:38:00,1,"29,86","10,45","3,88","15,53"
3,578,36909N,Crest Joggers,M,1,2021-05-08 15:54:00,34219,UK,"4,40",Black,...,Women,8,5,2021,15:54:00,5,"28,50","9,98","3,71","14,82"
4,863,53352X,Crew Socks 5pk,M,2,2021-03-27 22:38:00,17438,USA,"4,70",Black,...,Accesorios,27,3,2021,22:38:00,5,"29,86","10,45","3,88","15,53"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,962809,13003A,Everyday Holdall Medium,S,3,2023-01-22 15:53:00,37575,UK,"4,80",Cherry Brown/Athletic Maroon,...,Accesorios,22,1,2023,15:53:00,6,"173,25","60,64","22,52","90,09"
409996,313539,30584H,Elevate Leggings,M,3,2023-06-08 17:09:00,6728,Canada,"5,00",Vanilla Brown Marl,...,Women,8,6,2023,17:09:00,3,"216,30","75,70","28,12","112,48"
409997,288199,57834W,Rest Day Essentials T-Shirt,S,3,2023-11-25 00:00:00,6940,UK,"4,70",Baked Maroon/Salsa Red,...,Women,25,11,2023,00:00:00,5,"80,64","28,22","10,48","41,93"
409998,91653,22861Z,"Sport 7"" Shorts",S,2,2023-11-16 09:09:00,69998,USA,"5,00",Plum Pink,...,Women,16,11,2023,09:09:00,3,"61,60","21,56","8,01","32,03"


In [64]:
sales_gymshark.to_csv('sales_gymshark_clean2.csv', index=False)
sales_gymshark.to_csv('sales_gymshark_PB.csv', index=False)

### DF Nº 2: CUSTOMERS 2023

In [18]:
# Importo el CSV de CUSTOMER resultante del fake
customer = pd.read_csv('customer.csv')

In [19]:
len(customer)

100000

In [20]:
customer.shape

(100000, 6)

In [21]:
customer.columns

Index(['CustomerID', 'Name', 'Lastname', 'Age', 'Gender', 'Country'], dtype='object')

In [22]:
customer.isnull().sum()

CustomerID    0
Name          0
Lastname      0
Age           0
Gender        0
Country       0
dtype: int64

In [23]:
customer.duplicated().sum()

0